In [102]:
import pandas as pd
from pyspark.sql import SparkSession
import numpy as np
from pyspark.sql.functions import col, udf, regexp_replace
from pyspark.sql.types import IntegerType, DoubleType

spark = SparkSession.builder.appName("FIFA").getOrCreate()

In [103]:
df = spark.read.csv('FIFA19.csv', header=True)

In [104]:
df = df.withColumn('ID', col('ID').cast('Integer')) \
       .withColumn('Name', col('Name').cast('String')) \
       .withColumn('Age', col('Age').cast('Integer')) \
       .withColumn('Nationality', col('Nationality').cast('String')) \
       .withColumn('Overall', col('Overall').cast('Integer')) \
       .withColumn('Potential', col('Potential').cast('Integer')) \
       .withColumn('Club', col('Club').cast('String')) \
       .withColumn('Value', col('Value').cast('String')) \
       .withColumn('Wage', col('Wage').cast('String')) \
       .withColumn('Preferred Foot', col('Preferred Foot').cast('String')) \
       .withColumn('Body Type', col('Body Type').cast('String')) \
       .withColumn('Position', col('Position').cast('String')) \
       .withColumn('Height', col('Height').cast('String')) \
       .withColumn('Weight', col('Weight').cast('String'))
       
df = df.select("ID",
               "Name", 
               "Age", 
               "Nationality",
               "Overall", 
               "Club",
               "Value", 
               "Wage",
               "Preferred Foot",
               "Weak Foot",
               "Body Type",
               "Position", 
               "Jersey Number", 
               "Joined",
               "Height", 
               "Weight",
               "Crossing", 
               "Finishing", 
               "HeadingAccuracy", 
               "ShortPassing", 
               "Volleys", 
               "Dribbling", 
               "Curve", 
               "FKAccuracy", 
               "LongPassing", 
               "BallControl", 
               "Acceleration", 
               "SprintSpeed", 
               "Agility", 
               "Reactions", 
               "Balance", 
               "ShotPower", 
               "Jumping", 
               "Stamina", 
               "Strength", 
               "LongShots", 
               "Aggression", 
               "Interceptions", 
               "Positioning", 
               "Vision", 
               "Penalties", 
               "Composure", 
               "Marking", 
               "StandingTackle", 
               "SlidingTackle", 
               "GKDiving", 
               "GKHandling", 
               "GKKicking", 
               "GKPositioning", 
               "GKReflexes", 
               "Release Clause")

In [105]:
def convert_money(Value):
    if Value is None:
        return None
    Value = Value.replace('€', '')
    if 'K' in Value:
        return int(float(Value.replace('K', '')) * 1000)
    elif 'M' in Value:
        return int(float(Value.replace('M', '')) * 1000000)
    else:
        return int(Value)

convert_money_udf = udf(convert_money, IntegerType())

def convert_height(height):
    if height is None:
        return None
    feet, inches = height.split("'")
    feet = int(feet)
    inches = int(inches.replace('"', ''))
    total_inches = feet * 12 + inches
    return round(total_inches * 2.54, 2) 

convert_height_udf = udf(convert_height, DoubleType())

In [106]:
df = df.withColumn('Value', convert_money_udf(col('Value'))) \
       .withColumn('Wage', convert_money_udf(col('Wage'))) \
       .withColumn('Weight', regexp_replace(col('Weight'), 'lbs', '')) \
       .withColumn('Height', convert_height_udf(col('Height'))) \
       .withColumn('Release Clause', convert_money_udf(col('Release Clause')))

In [107]:
df.show()

+-----+---------------+---+-------------------+-------+--------------------+--------+------+--------------+---------+---------+--------+-------------+------------+------+------+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+--------------+
|   ID|           Name|Age|        Nationality|Overall|                Club|   Value|  Wage|Preferred Foot|Weak Foot|Body Type|Position|Jersey Number|      Joined|Height|Weight|Crossing|Finishing|HeadingAccuracy|ShortPassing|Volleys|Dribbling|Curve|FKAccuracy|LongPassing|BallControl|Acceleration|SprintSpeed|Agility|Reactions|Balance|ShotPower|Jumping|Stamina|Strength|LongShots|Aggression|Interceptions|Positioning|Vision|Penalties|Composure|Mark